In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(graularity):
    value = ['2018-05-06', '2018-05-12']
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'weekly'
value = ['2018-05-06', '2018-05-12']
check_store_unified_db_completeness(key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'monthly'
value = ['2018-05-01', '2018-05-31']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'quarterly'
value = ['2018-04-01', '2018-06-30']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'yearly'
value = ['2018-01-01', '2018-12-31']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'weekly'
value = ['2020-01-26', '2020-02-01']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'monthly'
value = ['2020-02-01', '2020-02-29']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = ''
value = ['2020-01-01', '2020-03-31']
check_store_unified_db_completeness(value, key)

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_q_fact_v1 
        where date between '2019-03-31' and '2019-03-31' and granularity='quarterly';
EOF


In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'weekly'
value = ['2020-02-02', '2020-02-08']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'weekly'
value = ['2019-12-29', '2020-01-04']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    db_result = query(aa_dsn, sql.format(graularity[0], date, date, graularity))
    # print date
    # print unified_result[0][0]
    # print db_result
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'monthly'
value = ['2020-01-01', '2020-01-31']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    quarterly_est_bucket= "store.app-est.v3"
    category_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
    "{}/fact/".format(quarterly_est_bucket)).where("granularity='{}' and date between '{}' and '{}'".format(graularity, end, end))
    category_df.createOrReplaceTempView("category_daily_df")
    db_result = spark.sql("select count(1), sum(free_app_download), sum(paid_app_download), sum(revenue) from category_daily_df").collect()
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'weekly'
value = ['2020-03-01', '2020-03-07']
check_store_unified_db_completeness(value, key)

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
daily_sql = """select count(1), 
            sum(est_free_app_download), 
            sum(est_paid_app_download), 
            sum(est_revenue)
            from 
    (select app_id, device_code, country_code,
            sum(est_free_app_download) as est_free_app_download, 
            sum(est_paid_app_download) as est_paid_app_download,
            sum(est_revenue) as est_revenue
            from store.store_est_fact_v1
            where date between '{}' and '{}'
            group by
            app_id,
            device_code,
            country_code) as t;
"""

sql = """select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_t_{}_fact_v1 
        where date between '{}' and '{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def check_store_unified_db_completeness(value, graularity):
    start = value[0]
    end = value[1]
    date = end
    unified_result = query(aa_dsn, daily_sql.format(start, end))
    quarterly_est_bucket= "store.app-est.v3"
    category_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
    "{}/fact/".format(quarterly_est_bucket)).where("granularity='{}' and date between '{}' and '{}'".format(graularity, end, end))
    category_df.createOrReplaceTempView("category_daily_df")
    db_result = spark.sql("select count(1), sum(free_app_download), sum(paid_app_download), sum(revenue) from category_daily_df").collect()
    if unified_result[0][0] == db_result[0][0] and unified_result[0][1] == db_result[0][1] and unified_result[0][2] == db_result[0][2] and unified_result[0][3] == db_result[0][3]:
        print "Completeness Test PASS! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])
    else:
        test_result.append(date)
        print "Completeness Test FAIL!!!!! date: {}, unified: {}, db: {}".format(date, unified_result[0], db_result[0])



key = 'monthly'
value = ['2020-03-01', '2020-03-31']
check_store_unified_db_completeness(value, key)

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select count(1), 
        sum(est_free_app_download), 
        sum(est_paid_app_download), 
        sum(est_revenue)
        from store.store_est_category_t_w_fact_v1 
        where date between '2015-11-07' and '2015-11-07' and granularity='weekly';
EOF

In [0]:
%%sh
